# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


## step 1
Installing dependencies 

In [17]:
!pip install beautifulsoup4
!pip install geocoder
!pip install folium
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium

     |████████████████████████████████| 92kB 208kB/s 


Scraping required data from webpage

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='wikitable')
table_rows = table.find_all('tr')

converting scraped data into padas dataframe

In [3]:
# this array will hold the table data
temp = []

# adding invidual subarrays for each table array
for tr in table_rows:
    td = tr.find_all('td')
    row = [d.text.strip() for d in td]

    # excluding rows whose borough is not assigned
    if row and row[1] != 'Not assigned':
        # in case post code don't have neighbourhood assigned
        if row[2] == 'Not assigned':
            row[2] = row[1]
        temp.append(row)

converting array with stored data into dataframe

In [20]:
# creating dataframe out of mentioned array
df = pd.DataFrame(data=temp, columns=['Postcode', 'Borough', 'Neighbourhood'])
print(df.shape)
# creating temporary data frame of neighbourhood seprated by comma's for each postcode
tempNeighbourhoodDf =df.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index().set_index('Postcode')

#droping rows and joining above dataframes 
df = df.drop('Neighbourhood', axis=1).drop_duplicates().set_index('Postcode')
df = df.join(tempNeighbourhoodDf)
print(df)

(211, 3)
                   Borough                                      Neighbourhood
Postcode                                                                     
M3A             North York                                          Parkwoods
M4A             North York                                   Victoria Village
M5A       Downtown Toronto                           Harbourfront,Regent Park
M6A             North York                    Lawrence Heights,Lawrence Manor
M7A           Queen's Park                                       Queen's Park
...                    ...                                                ...
M8X              Etobicoke        The Kingsway,Montgomery Road,Old Mill North
M4Y       Downtown Toronto                               Church and Wellesley
M7Y           East Toronto  Business Reply Mail Processing Centre 969 Eastern
M8Y              Etobicoke  Humber Bay,King's Mill Park,Kingsway Park Sout...
M8Z              Etobicoke  Kingsway Park South West,Mi

### shape of final dataframe    

In [21]:
# i have made Postcode as my index, that's why i get two rows count in shape of column
print("Shape of final data frame", df.shape)

Shape of final data frame (103, 2)


## step 2: getting Latitudes & Longitudes

In [6]:
import geocoder

def fetchLatLng(postal_code):

    # iniitalize your variable to none
    lat_lng_words = None

    # loop until you get coordinates
    while(lat_lng_words is None):
        g = geocoder.google('{}, Toronto, Ontaria'.format(postal_code))
        lat_lng_words = g.latlng

    latitude = lat_lng_words[0]
    longitude = lat_lng_words[1]
    print('Latitude: {} & longitude: {}'.format(latitude, longitude))
    return

In [7]:
# fetchLatLng('M3A')

### getting issue with fetching longitude so i will be using provided data of location

In [22]:
locationDf = pd.read_csv('./Geospatial_Coordinates.csv').set_index('Postal Code')
df = df.join(locationDf)
torontoDf = df[df['Borough'].str.contains("Toronto")]
torontoDf

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
M4E,East Toronto,The Beaches,43.676357,-79.293031
M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
M6G,Downtown Toronto,Christie,43.669542,-79.422564
M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259


Getting all rows of dataframe where "Toronto" comes in Borough in name

In [28]:
# geocoder is not working in my case
# getting lat, lng of Toronto
lat_lng_toronto = None

# loop until you get coordinates
while(lat_lng_toronto is None):
    print("fetching location")
    g = geocoder.google('Toronto, Ontaria')
    print("fetched location")
    lat_lng_toronto = g.latlng

lat_lng_toronto

fetching location


In [0]:
 print(lat_lng_toronto)

In [27]:
# plotting map of toronto borough
Citoronto_latitude = "43.6529"
toronto_longitude = "-79.3849"
map_toronto = folium. Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

for lat, lng, borough, neighbourhood in zip(torontoDf['Latitude'], torontoDf['Longitude'], torontoDf['Borough'], torontoDf['Neighbourhood']):
    label = '{}'.format(borough)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

map_toronto